# **CUHK-STAT3009**: Homework 2 - Matrix Facorization: BCD or ALS **(due Oct 26)**

-   Note that we use the same notations in the STAT3009 slides


In [1]:
# Load and pro-processed dataset
import numpy as np
import pandas as pd

## Upload Netflix dataset in CUHK-STAT3009 Github repo

train_url = (
    "https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/train.csv"
)
test_url = (
    "https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/test.csv"
)

dtrain = pd.read_csv(train_url)
dtest = pd.read_csv(test_url)

train_rating = dtrain["rating"].values
train_rating = np.array(train_rating, dtype=float)
train_pair = dtrain[["user_id", "movie_id"]].values

test_rating = dtest["rating"].values
test_rating = np.array(test_rating, dtype=float)
test_data = dtest[["user_id", "movie_id"]].values

n_user = max(max(train_pair[:, 0]), max(test_data[:, 0])) + 1
n_item = max(max(train_pair[:, 1]), max(test_data[:, 1])) + 1


def rmse(test_rating, pred_rating):
    return np.sqrt(np.mean((pred_rating - test_rating) ** 2))

### Q1: User-item average based RS

Build up a recommender system where the predicted rating is given as the _average of user-mean and item-mean_, that is:

$$
\widehat{r}_{ui} = (\bar{r}_u + \bar{r}_i) / 2,
$$

where $\bar{r}_u = \frac{1}{|\mathcal{I}_u|} \sum_{i \in \mathcal{I}_u} r_{ui}$ and $\bar{r}_i = \frac{1}{|\mathcal{U}_i|} \sum_{u \in \mathcal{U}_i} r_{ui}$.

-   Compute RMSE for your recommender system
-   Implement the RS by a `skikit-learn` compatible `class` with `fit` and `predict`
-   **(MUST-DO-STEP)** Print the `test_pair`, `pred_rating`, and `true_rating` for the T-th record in the `test.csv`, where T is the last four digits of your student Id. For example, if your `student Id =  1155111111`, please print the 1111-th record.


In [2]:
## Your solution here
class UserMean:
    def __init__(self) -> None:
        self.users = {}
        self.global_mean = 0

    def fit(self, data: np.array, target: np.array) -> None:
        for (user_id, item_id), rating in zip(data, target):
            self.users[user_id] = np.append(
                self.users.get(user_id, np.array([])), rating
            )

        self.global_mean = np.mean(target)
        return self

    def predict(self, data: np.array):
        result = np.array([])
        for user_id, item_id in data:
            result = np.append(
                result, self.users.get(user_id, np.array([self.global_mean])).mean()
            )
        return result


class ItemMean:
    def __init__(self) -> None:
        self.items = {}
        self.global_mean = 0

    def fit(self, data: np.array, target: np.array) -> None:
        for (user_id, item_id), rating in zip(data, target):
            self.items[item_id] = np.append(
                self.items.get(item_id, np.array([])), rating
            )

        self.global_mean = np.mean(target)
        return self

    def predict(self, data: np.array):
        result = np.array([])
        for user_id, item_id in data:
            result = np.append(
                result, self.items.get(item_id, np.array([self.global_mean])).mean()
            )
        return result


class CustomRS:
    def __init__(self) -> None:
        self.user_mean = UserMean()
        self.item_mean = ItemMean()

    def fit(self, data: np.array, target: np.array):
        self.user_mean.fit(data.copy(), target.copy())
        self.item_mean.fit(data.copy(), target.copy())
        return self

    def predict(self, data: np.array):
        user_mean_prediction = self.user_mean.predict(data.copy())
        item_mean_prediction = self.item_mean.predict(data.copy())
        return (user_mean_prediction + item_mean_prediction) / 2


custom_rs = CustomRS().fit(train_pair, train_rating)
test_prediction = custom_rs.predict(test_data)
print(f"RMSE of custom RS: {rmse(test_rating, test_prediction)}")
print("================================================================")

t = 8560
tth_test_pair = test_data[t]
tth_pred_rating = custom_rs.predict(np.array([tth_test_pair]))[0]
tth_true_rating = test_rating[t]

print(f"{tth_test_pair=}")
print(f"{tth_pred_rating=}")
print(f"{tth_true_rating=}")

RMSE of custom RS: 0.981602889281252
tth_test_pair=array([1776, 1931])
tth_pred_rating=4.524252491694352
tth_true_rating=5.0


## Q2: `Lasso-MF` Recommender systems

### **Background**

Given the training data $(\mathbf{x}_i, y_i)_{i=1}^n$, where $\mathbf{x}_i$ is the feature-vector, $y_i$ is the ground truth score. Lasso regression considers following formulation for a sparse solution [see ref](https://www.statisticshowto.com/lasso-regression/):

$$
\text{argmin}_{\mathbf{\beta}} \ \frac{1}{n} \sum_{i=1}^n ( y_i - \mathbf{\beta}^T \mathbf{x}_i )^2 + \lambda \| \mathbf{\beta} \|_1, \quad \text{where } \| \mathbf{\beta} \|_1 = \sum_{j=1}^p |\beta_j|,
$$

which can be solved by `sklearn.linear_model.Lasso` ([doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)), and a usage example is provided in [here](https://scikit-learn.org/stable/modules/linear_model.html#lasso).

### **Your Tasks**

Build up `Lasso-MF` RS by solving the following formulation:

$$
(\widehat{\mathbf P}, \widehat{\mathbf Q}) = \text{argmin}_{\mathbf{P}, \mathbf{Q} } \frac{1}{|\Omega|} \sum_{(u,i) \in \Omega} ( r_{ui} - \mathbf{p}^\intercal_u \mathbf{q}_i  )^2 + \lambda \big(  \sum_{u=1}^n \|\mathbf{p}_u\|_1 + \sum_{i=1}^m \|\mathbf{q}_i\|_1 \big). \qquad (1)
$$

-   Implement `Lasso-MF` with a class `Lasso_MF` contains two methods: `Lasso_MF.fit`, `Lasso_MF.predict`. The parameters $\mathbf{P}$ and $\mathbf{Q}$ should be fitted by (1) based on `Lasso_MF.fit` method, and the rating is predicted by `Lasso_MF.predict` based on
    $$
    \widehat{r}_{ui} = \widehat{\mathbf{p}}^T_u \widehat{\mathbf{q}}_i
    $$

(_Hint_) Using the same logic of Alternative Least Square (ALS): each subproblem is a Lasso regression which can be solved by `sklearn.linear_model.Lasso`.

-   Print `RMSE` for testing data based on $(\lambda = 0.1, K = 3)$ and $(\lambda = 0.3, K = 5)$,
-   (**MUST-DO-STEP**) Print the `test_pair`, `pred_rating`, and `true_rating` for the T-th record in the testing data, where T is the last four digits of your student Id. For example, if your `student Id =  1155111111`, please print the 1111-th record.


In [3]:
## Your solution here
from sklearn.linear_model import Lasso

import math


class Lasso_MF:
    def __init__(
        self,
        n_user: int,
        n_item: int,
        lambda_: float,
        k: int,
        max_iteration=50,
        tolerance: float | None = None,
        lasso_max_iter: int = 1000,
        lasso_tol: float = 1e-4,
        verbose: int = 0,
    ) -> None:
        self.n_user = n_user
        self.n_item = n_item
        self.P = np.random.randn(n_user, k)
        self.Q = np.random.randn(n_item, k)
        self.lambda_ = lambda_
        self.k = k
        self.max_iter = max_iteration
        self.tolerance = tolerance
        self.lasso_max_iter = lasso_max_iter
        self.lasso_tol = lasso_tol
        self.verbose = verbose

    def fit(self, data: np.array, target: np.array):
        user_records = {
            user: np.where(data[:, 0] == user)[0] for user in range(self.n_user)
        }
        item_records = {
            item: np.where(data[:, 1] == item)[0] for item in range(self.n_item)
        }
        previous_objective = math.inf
        for iteration in range(self.max_iter):
            # item update
            for item in range(self.n_item):
                item_record_id = item_records[item]
                if len(item_record_id):
                    lasso = Lasso(
                        self.lambda_,
                        fit_intercept=False,
                        max_iter=self.lasso_max_iter,
                        tol=self.lasso_tol,
                    )
                    lasso.fit(
                        self.P[data[item_record_id][:, 0]], target[item_record_id]
                    )
                    self.Q[item, :] = np.array(lasso.coef_)

            # user update
            for user in range(self.n_user):
                user_record_id = user_records[user]
                if len(user_record_id):
                    lasso = Lasso(
                        self.lambda_,
                        fit_intercept=False,
                        max_iter=self.lasso_max_iter,
                        tol=self.lasso_tol,
                    )
                    lasso.fit(
                        self.Q[data[user_record_id][:, 1]], target[user_record_id]
                    )
                    self.P[user, :] = np.array(lasso.coef_)

            current_objective = self.objective(data, target)
            objective_difference = (
                (abs(previous_objective - current_objective) / previous_objective)
                if previous_objective != math.inf
                else 1.0
            )
            if self.verbose != 0:
                print(
                    f"Iteration {iteration+1:>4d}| Objective function: {current_objective:>10.7f} | Difference: {objective_difference:>6.5f}"
                )

            if self.tolerance is not None and self.tolerance > objective_difference:
                print(
                    f"Objective difference({objective_difference:>6.5f}) smaller than tolerance({self.tolerance:>6.5f}). Terminating training..."
                )
                break
            previous_objective = current_objective
        return self

    def predict(self, test_data):
        return np.array([np.dot(self.P[row[0]], self.Q[row[1]]) for row in test_data])

    def objective(self, data, target):
        return rmse(self.predict(data), target) ** 2 + self.lambda_ * (
            np.sum(np.abs(self.P)) + np.sum(np.abs(self.Q))
        )


t = 8560
tth_test_pair = test_data[t]
tth_true_rating = test_rating[t]

lambda_, k = 0.1, 3
lasso_mf1 = Lasso_MF(
    n_user,
    n_item,
    k=k,
    lambda_=lambda_,
    verbose=1,
    lasso_max_iter=100000,
    lasso_tol=0.01,
).fit(train_pair, train_rating)
test_prediction = lasso_mf1.predict(test_data)
tth_pred_rating = lasso_mf1.predict(np.array([tth_test_pair]))[0]
print(f"RMSE of LassoMF(lambda={lambda_}, {k=}): {rmse(test_rating, test_prediction)}")
print(f"{tth_test_pair=}")
print(f"{tth_pred_rating=}")
print(f"{tth_true_rating=}")
print("================================================================")

lambda_, k = 0.3, 5
lasso_mf2 = Lasso_MF(
    n_user,
    n_item,
    k=k,
    lambda_=lambda_,
    verbose=1,
    lasso_max_iter=100000,
    lasso_tol=0.01,
).fit(train_pair, train_rating)
test_prediction = lasso_mf2.predict(test_data)
tth_pred_rating = lasso_mf1.predict(np.array([tth_test_pair]))[0]
print(f"RMSE of LassoMF(lambda={lambda_}, {k=}): {rmse(test_rating, test_prediction)}")
print(f"{tth_test_pair=}")
print(f"{tth_pred_rating=}")
print(f"{tth_true_rating=}")
print("================================================================")

Iteration    1| Objective function: 2762.8914430 | Difference: 1.00000
Iteration    2| Objective function: 2539.3994874 | Difference: 0.08089
Iteration    3| Objective function: 1928.7517388 | Difference: 0.24047
Iteration    4| Objective function: 1654.6205084 | Difference: 0.14213
Iteration    5| Objective function: 1516.6598692 | Difference: 0.08338
Iteration    6| Objective function: 1426.3441705 | Difference: 0.05955
Iteration    7| Objective function: 1358.6431304 | Difference: 0.04746
Iteration    8| Objective function: 1314.1184220 | Difference: 0.03277
Iteration    9| Objective function: 1280.5017964 | Difference: 0.02558
Iteration   10| Objective function: 1256.9421368 | Difference: 0.01840
Iteration   11| Objective function: 1241.0620356 | Difference: 0.01263
Iteration   12| Objective function: 1228.9771813 | Difference: 0.00974
Iteration   13| Objective function: 1217.6822062 | Difference: 0.00919
Iteration   14| Objective function: 1206.9362998 | Difference: 0.00882
Iterat

### Q3 (Bonus): BCD for baseline + MF

Buid up a `user average` + `item average` + `MF` based recommender system, where the predicted rating is given as:

$$
\widehat{r}_{ui} = \mu_u + \rho_i + \mathbf{p}_u^T \mathbf{q}_i,
$$

where $(\mu_u, \mathbf{p}_u); (u=1, \cdots, n)$ and $(\rho_i, \mathbf{q}_i); (i=1,\cdots,m)$ are fitted by:

$$
 \min_{\mathbf{\mu}, \mathbf{\rho}, \mathbf{P}, \mathbf{Q} } \frac{1}{|\Omega|} \sum_{(u,i) \in \Omega} ( r_{ui} - \mu_u - \rho_i - \mathbf{p}^\intercal_u \mathbf{q}_i  )^2 + \lambda \big(  \sum_{u=1}^n \|\mathbf{p}_u\|_2^2 + \sum_{i=1}^m \|\mathbf{q}_i\|_2^2 \big).
$$

-   3.1: Find the BCD solution for $(\mu_u)_{u=1}^n$, $(\rho_i)_{i=1}^m$, $\mathbf{P}$ and $\mathbf{Q}$ at one iteration

-   3.2: Implement the RS by a `skikit-learn` compatible `class` with `fit` and `predict`

    -   Print `rmse` for the model with $(\lambda=.1, K=3)$
    -   (**MUST-DO-STEP**) Print the `test_pair`, `pred_rating`, and `true_rating` for the T-th record in the testing data, where T is the last four digits of your student Id. For example, if your `student Id =  1155111111`, please print the 1111-th record.


#### Your solution to Q3.1 here

**Note:** you can use [Latex](https://latex-tutorial.com/tutorials/amsmath/) to add the solution, or your can add your solution by uploading an image

At l-th iteration:

-   By fixing other, $(\mu_u)_{u=1}^n$ are updated as:

    $$
    \hat{\mu}^{(l+1)}_u = \frac{1}{|\mathbf{I}_{u}|} \sum_{i \in \mathbf{I}_{u}} (r_{ui} - \rho_i -\mathbf{p}^\intercal_u \mathbf{q}_i)
    $$

-   By fixing other, $(\rho_i)_{i=1}^m$ are updated as:

    $$
    \hat{\rho}^{(l+1)}_i = \frac{1}{|\mathbf{U}_{i}|} \sum_{i \in \mathbf{U}_{i}} (r_{ui} - \mu_u -\mathbf{p}^\intercal_u \mathbf{q}_i)
    $$

-   By fixing other, $(\mathbf{p}_u)_{u=1}^n$ are updated as:

    $$
    \hat{\mathbf{p}}^{(l+1)}_u = (\sum_{i \in \mathbf{I}_{u}} (\mathbf{q}_i \mathbf{q}^\intercal_i + \lambda |\Omega| \mathbf{I}))^{-1} \sum_{i \in \mathbf{I}_{u}} (r_{ui} - \mu_u - \rho_i ) \mathbf{q}_i
    $$

-   By fixing other, $(\mathbf{q}_i)_{i=1}^m$ are updated as:
    $$
    \hat{\mathbf{p}}^{(l+1)}_u = (\sum_{u \in \mathbf{U}_{i}} (\mathbf{p}_u \mathbf{p}^\intercal_u + \lambda |\Omega| \mathbf{I}))^{-1} \sum_{u \in \mathbf{U}_{i}} (r_{ui} - \mu_u - \rho_i ) \mathbf{p}_u
    $$


In [4]:
## Your solution to Q3.2 here
import math
from tqdm import tqdm


class AveragesMF:
    def __init__(
        self,
        n_user: int,
        n_item: int,
        lambda_: float,
        k: int,
        max_iteration=50,
        tolerance: float | None = None,
        verbose: int = 0,
    ) -> None:
        self.n_user = n_user
        self.n_item = n_item
        self.user_average = list(np.random.randn(n_user, 1))
        self.item_average = list(np.random.randn(n_item, 1))
        self.P = np.random.randn(n_user, k)
        self.Q = np.random.randn(n_item, k)
        self.lambda_ = lambda_
        self.k = k
        self.max_iter = max_iteration
        self.tolerance = tolerance
        self.verbose = verbose

    def fit(self, data: np.array, target: np.array):
        user_records = {
            user: np.where(data[:, 0] == user)[0] for user in range(self.n_user)
        }
        item_records = {
            item: np.where(data[:, 1] == item)[0] for item in range(self.n_item)
        }
        n_obs = len(data)
        previous_objective = math.inf
        for iteration in range(self.max_iter):
            # user_average update
            # for user in (progress_bar := tqdm(range(self.n_user))):
            #     progress_bar.set_description(f"Finding user mean")
            for user in range(self.n_user):
                user_record_id = user_records[user]
                if len(user_record_id):
                    residuals = np.zeros_like(user_record_id)
                    for residual_index, (item, rating) in enumerate(
                        zip(data[user_record_id][:, 1], target[user_record_id])
                    ):
                        residuals[residual_index] = (
                            rating
                            - self.item_average[item]
                            - np.dot(self.P[user], self.Q[item])
                        )
                    self.user_average[user] = np.mean(residuals)

            # item_average update
            # for item in (progress_bar := tqdm(range(self.n_item))):
            #     progress_bar.set_description(f"Finding item mean")
            for item in range(self.n_item):
                item_record_id = item_records[item]
                if len(item_record_id):
                    residuals = np.zeros_like(item_record_id)
                    for residual_index, (user, rating) in enumerate(
                        zip(data[item_record_id][:, 0], target[item_record_id])
                    ):
                        residuals[residual_index] = (
                            rating
                            - self.user_average[user]
                            - np.dot(self.P[user], self.Q[item])
                        )
                    self.item_average[item] = np.mean(residuals)

            # user update
            # for user in (progress_bar := tqdm(range(self.n_user))):
            #     progress_bar.set_description(f"Finding P_u")
            for user in range(self.n_user):
                user_record_id = user_records[user]
                if len(user_record_id) == 0:
                    self.P[user_record_id, :] = 0.0
                    continue

                sum_qi, sum_matrix = np.zeros((self.k)), np.zeros((self.k, self.k))
                for record_id in user_record_id:
                    item, rating_tmp = train_pair[record_id, 1], train_rating[record_id]
                    sum_matrix = sum_matrix + np.outer(self.Q[item, :], self.Q[item, :])
                    sum_qi += (
                        rating_tmp - self.user_average[user] - self.item_average[item]
                    ) * self.Q[item, :]
                self.P[user, :] = (
                    np.linalg.inv(
                        sum_matrix + self.lambda_ * n_obs * np.identity(self.k)
                    )
                    @ sum_qi
                )

            # item update
            # for item in (progress_bar := tqdm(range(self.n_item))):
            #     progress_bar.set_description(f"Finding Q_i")
            for item in range(self.n_item):
                item_record_id = item_records[item]
                if len(item_record_id) == 0:
                    self.P[item_record_id, :] = 0.0
                    continue

                sum_qi, sum_matrix = np.zeros((self.k)), np.zeros((self.k, self.k))
                for record_id in item_record_id:
                    user, rating_tmp = train_pair[record_id, 0], train_rating[record_id]
                    sum_matrix = sum_matrix + np.outer(self.Q[user, :], self.Q[user, :])
                    sum_qi += (
                        rating_tmp - self.user_average[user] - self.item_average[item]
                    ) * self.P[user, :]
                self.P[user, :] = (
                    np.linalg.inv(
                        sum_matrix + self.lambda_ * n_obs * np.identity(self.k)
                    )
                    @ sum_qi
                )

            current_objective = self.objective(data, target)
            objective_difference = (
                (abs(previous_objective - current_objective) / previous_objective)
                if previous_objective != math.inf
                else 1.0
            )
            if self.verbose != 0:
                print(
                    f"Iteration {iteration+1:>4d}| Objective function: {current_objective:>10.7f} | Difference: {objective_difference:>6.5f}"
                )

            if self.tolerance is not None and self.tolerance > objective_difference:
                print(
                    f"Objective difference({objective_difference:>6.5f}) smaller than tolerance({self.tolerance:>6.5f}). Terminating training..."
                )
                break
            previous_objective = current_objective

        return self

    def predict(self, test_data):
        return np.array(
            [
                self.user_average[user]
                + self.item_average[item]
                + np.dot(self.P[user], self.Q[item])
                for user, item in test_data
            ]
        )

    def objective(self, data, target):
        return rmse(self.predict(data), target) ** 2 + self.lambda_ * (
            np.sum(self.P**2) + np.sum(self.Q**2)
        )


lambda_, k = 0.1, 3
avg_mf = AveragesMF(n_user=n_user, n_item=n_item, lambda_=lambda_, k=k, verbose=1).fit(
    train_pair, train_rating
)
t = 8560
tth_test_pair = test_data[t]
tth_true_rating = test_rating[t]
tth_pred_rating = avg_mf.predict(np.array([tth_test_pair]))[0]
print(
    f"RMSE of User mean + Item mean + MF(lambda={lambda_}, {k=}): {rmse(test_rating, test_prediction)}"
)
print(f"{tth_test_pair=}")
print(f"{tth_pred_rating=}")
print(f"{tth_true_rating=}")
print("================================================================")

Iteration    1| Objective function: 1102.3337616 | Difference: 1.00000
Iteration    2| Objective function: 1102.1724729 | Difference: 0.00015
Iteration    3| Objective function: 1102.2089065 | Difference: 0.00003
Iteration    4| Objective function: 1102.2318265 | Difference: 0.00002
Iteration    5| Objective function: 1102.2317462 | Difference: 0.00000
Iteration    6| Objective function: 1102.2357602 | Difference: 0.00000
Iteration    7| Objective function: 1102.2411078 | Difference: 0.00000
Iteration    8| Objective function: 1102.2465093 | Difference: 0.00000
Iteration    9| Objective function: 1102.2496229 | Difference: 0.00000
Iteration   10| Objective function: 1102.2499932 | Difference: 0.00000
Iteration   11| Objective function: 1102.2503151 | Difference: 0.00000
Iteration   12| Objective function: 1102.2492672 | Difference: 0.00000
Iteration   13| Objective function: 1102.2552146 | Difference: 0.00001
Iteration   14| Objective function: 1102.2578213 | Difference: 0.00000
Iterat

## Q4 (**Bonus**): **Parallel** ALS for MF

### **Background**

Recall the item/user updates in Algo 2 (in the slides):
$$ \mathbf{q}^{(l+1)}_i = ( \sum_{u \in \mathcal{U}_i} \mathbf{p}^{(l)}\_u (\mathbf{p}^{(l)}\_u)^T + \lambda |\Omega| \mathbf{I})^{-1} \sum_{u \in \mathcal{U}_i} r_{ui} \mathbf{p}^{(l)}_u,$$
$$\mathbf{p}^{(l+1)}\_u = ( \sum_{i \in \mathcal{I}_u} \mathbf{q}^{(l+1)}\_i (\mathbf{q}^{(l+1)}\_i)^\intercal + \lambda |\Omega| \mathbf{I})^{-1} \sum_{i \in \mathcal{I}_u} r_{ui} \mathbf{q}^{(l+1)}\_i.$$

-   The key observation is that the update for user-u/item-i is independent with other users/items. Therefore, they can be conducted parallelly.
-   Suppose your have 100 users to update, the basic ALS updates user 1, user 2, ... user 100 one by one in the loop. Now, suppose you have 100 CPUs, the **parallel ALS** can update 100 users _simultaneously_ by putting each user to different CPUs, which significantly reduce the computation time.

### **Your tasks**

-   Use Python libraries [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) or [pymp](https://github.com/classner/pymp) to revise `MF.fit` method ([avaliable in In [16]](https://github.com/statmlben/CUHK-STAT3009/blob/main/nb_mf.ipynb)) allowing parallel updating of $\mathbf{p}_u$ and $\mathbf{q}_i$.

-   Compare the computation time for `MF.fit` with and w/o parallel computing by using `%%time` (see [ref](https://stackoverflow.com/questions/32565829/simple-way-to-measure-cell-execution-time-in-ipython-notebook)).


In [5]:
# check number of CPUs in your PC/Node
!sysctl -n hw.ncpu

12


In [2]:
# benchmark
class MF(object):
    def __init__(
        self, n_user, n_item, lam=0.001, K=10, iterNum=50, tol=1e-4, verbose=1
    ):
        self.P = np.random.randn(n_user, K)
        self.Q = np.random.randn(n_item, K)
        # self.index_item = []
        # self.index_user = []
        self.n_user = n_user
        self.n_item = n_item
        self.lam = lam
        self.K = K
        self.iterNum = iterNum
        self.tol = tol
        self.verbose = verbose

    def fit(self, train_pair, train_rating):
        diff, tol = 1.0, self.tol
        n_user, n_item, n_obs = self.n_user, self.n_item, len(train_pair)
        K, iterNum, lam = self.K, self.iterNum, self.lam
        ## store user/item index set
        self.index_item = [np.where(train_pair[:, 1] == i)[0] for i in range(n_item)]
        self.index_user = [np.where(train_pair[:, 0] == u)[0] for u in range(n_user)]

        if self.verbose:
            print("Fitting Reg-MF: K: %d, lam: %.5f" % (K, lam))

        for i in range(iterNum):
            ## item update
            obj_old = self.obj(test_pair=train_pair, test_rating=train_rating)
            for item_id in range(n_item):
                index_item_tmp = self.index_item[item_id]
                if len(index_item_tmp) == 0:
                    self.Q[item_id, :] = 0.0
                    continue
                ## compute `sum_pu` and `sum_matrix`
                sum_pu, sum_matrix = np.zeros((K)), np.zeros((K, K))
                for record_ind in index_item_tmp:
                    ## double-check
                    if item_id != train_pair[record_ind][1]:
                        raise ValueError("the item_id is worning in updating Q!")
                    user_id, rating_tmp = (
                        train_pair[record_ind][0],
                        train_rating[record_ind],
                    )
                    sum_matrix = sum_matrix + np.outer(
                        self.P[user_id, :], self.P[user_id, :]
                    )
                    sum_pu = sum_pu + rating_tmp * self.P[user_id, :]
                self.Q[item_id, :] = np.dot(
                    np.linalg.inv(sum_matrix + lam * n_obs * np.identity(K)), sum_pu
                )

            for user_id in range(n_user):
                index_user_tmp = self.index_user[user_id]
                if len(index_user_tmp) == 0:
                    self.P[user_id, :] = 0.0
                    continue
                ## compute `sum_qi` and `sum_matrix`
                sum_qi, sum_matrix = np.zeros((K)), np.zeros((K, K))
                for record_ind in index_user_tmp:
                    ## double-check
                    if user_id != train_pair[record_ind][0]:
                        raise ValueError("the user_id is worning in updating P!")
                    item_id, rating_tmp = (
                        train_pair[record_ind][1],
                        train_rating[record_ind],
                    )
                    sum_matrix = sum_matrix + np.outer(
                        self.Q[item_id, :], self.Q[item_id, :]
                    )
                    sum_qi = sum_qi + rating_tmp * self.Q[item_id, :]
                self.P[user_id, :] = np.dot(
                    np.linalg.inv(sum_matrix + lam * n_obs * np.identity(K)), sum_qi
                )
            # compute the new rmse score
            obj_new = self.obj(test_pair=train_pair, test_rating=train_rating)
            diff = abs(obj_new - obj_old) / obj_old
            if self.verbose:
                print("Reg-MF: ite: %d; diff: %.3f Obj: %.3f" % (i, diff, obj_new))
            if diff < tol:
                break

    def predict(self, test_pair):
        # predict ratings for user-item pairs
        pred_rating = [np.dot(self.P[line[0]], self.Q[line[1]]) for line in test_pair]
        return np.array(pred_rating)

    def rmse(self, test_pair, test_rating):
        # report the rmse for the fitted `MF`
        pred_rating = self.predict(test_pair=test_pair)
        return np.sqrt(np.mean((pred_rating - test_rating) ** 2))

    def obj(self, test_pair, test_rating):
        return (
            (self.rmse(test_pair, test_rating)) ** 2
            + self.lam * np.sum(self.P**2)
            + self.lam * np.sum(self.Q**2)
        )

In [23]:
## You solution here
import pymp


class MultiprocessMF(MF):
    def __init__(
        self,
        n_user,
        n_item,
        lam=0.001,
        K=10,
        iterNum=50,
        tol=1e-4,
        n_process=1,
        verbose=1,
    ):
        super().__init__(n_user, n_item, lam, K, iterNum, tol, verbose)
        self.n_process = n_process

    def fit(self, train_pair, train_rating):
        diff, tol = 1.0, self.tol
        n_user, n_item, n_obs = self.n_user, self.n_item, len(train_pair)
        K, iterNum, lam = self.K, self.iterNum, self.lam
        ## store user/item index set
        self.index_item = [np.where(train_pair[:, 1] == i)[0] for i in range(n_item)]
        self.index_user = [np.where(train_pair[:, 0] == u)[0] for u in range(n_user)]

        if self.verbose:
            print("Fitting Reg-MF: K: %d, lam: %.5f" % (K, lam))

        with pymp.Parallel(self.n_process) as p:
            for i in range(iterNum):
                ## item update
                obj_old = self.obj(test_pair=train_pair, test_rating=train_rating)
                for item_id in p.range(n_item):
                    index_item_tmp = self.index_item[item_id]
                    if len(index_item_tmp) == 0:
                        self.Q[item_id, :] = 0.0
                        continue
                    ## compute `sum_pu` and `sum_matrix`
                    sum_pu, sum_matrix = np.zeros((K)), np.zeros((K, K))
                    for record_ind in index_item_tmp:
                        ## double-check
                        if item_id != train_pair[record_ind][1]:
                            raise ValueError("the item_id is worning in updating Q!")
                        user_id, rating_tmp = (
                            train_pair[record_ind][0],
                            train_rating[record_ind],
                        )
                        sum_matrix = sum_matrix + np.outer(
                            self.P[user_id, :], self.P[user_id, :]
                        )
                        sum_pu = sum_pu + rating_tmp * self.P[user_id, :]
                    self.Q[item_id, :] = np.dot(
                        np.linalg.inv(sum_matrix + lam * n_obs * np.identity(K)), sum_pu
                    )

                for user_id in p.range(n_user):
                    index_user_tmp = self.index_user[user_id]
                    if len(index_user_tmp) == 0:
                        self.P[user_id, :] = 0.0
                        continue
                    ## compute `sum_qi` and `sum_matrix`
                    sum_qi, sum_matrix = np.zeros((K)), np.zeros((K, K))
                    for record_ind in index_user_tmp:
                        ## double-check
                        if user_id != train_pair[record_ind][0]:
                            raise ValueError("the user_id is worning in updating P!")
                        item_id, rating_tmp = (
                            train_pair[record_ind][1],
                            train_rating[record_ind],
                        )
                        sum_matrix = sum_matrix + np.outer(
                            self.Q[item_id, :], self.Q[item_id, :]
                        )
                        sum_qi = sum_qi + rating_tmp * self.Q[item_id, :]
                    self.P[user_id, :] = np.dot(
                        np.linalg.inv(sum_matrix + lam * n_obs * np.identity(K)), sum_qi
                    )
                # compute the new rmse score
                obj_new = self.obj(test_pair=train_pair, test_rating=train_rating)
                diff = abs(obj_new - obj_old) / obj_old
                if self.verbose:
                    print("Reg-MF: ite: %d; diff: %.3f Obj: %.3f" % (i, diff, obj_new))
                if diff < tol:
                    break

In [19]:
TOLERANCE = 0.000001

In [20]:
%%time
lambda_, k = 0.0001, 5
mf = MF(n_user, n_item, lam=lambda_, K=k, tol=TOLERANCE)
mf.fit(train_pair, train_rating)
test_prediction = mf.predict(test_data)
print(f"RMSE of MF(lambda={lambda_}, {k=}): {rmse(test_rating, test_prediction)}")

Fitting Reg-MF: K: 5, lam: 0.00010
Reg-MF: ite: 0; diff: 0.511 Obj: 10.855
Reg-MF: ite: 1; diff: 0.783 Obj: 2.360
Reg-MF: ite: 2; diff: 0.111 Obj: 2.098
Reg-MF: ite: 3; diff: 0.041 Obj: 2.012
Reg-MF: ite: 4; diff: 0.026 Obj: 1.960
Reg-MF: ite: 5; diff: 0.018 Obj: 1.925
Reg-MF: ite: 6; diff: 0.012 Obj: 1.903
Reg-MF: ite: 7; diff: 0.008 Obj: 1.889
Reg-MF: ite: 8; diff: 0.005 Obj: 1.880
Reg-MF: ite: 9; diff: 0.003 Obj: 1.874
Reg-MF: ite: 10; diff: 0.002 Obj: 1.870
Reg-MF: ite: 11; diff: 0.001 Obj: 1.868
Reg-MF: ite: 12; diff: 0.001 Obj: 1.866
Reg-MF: ite: 13; diff: 0.001 Obj: 1.865
Reg-MF: ite: 14; diff: 0.000 Obj: 1.864
Reg-MF: ite: 15; diff: 0.000 Obj: 1.863
Reg-MF: ite: 16; diff: 0.000 Obj: 1.863
Reg-MF: ite: 17; diff: 0.000 Obj: 1.862
Reg-MF: ite: 18; diff: 0.000 Obj: 1.862
Reg-MF: ite: 19; diff: 0.000 Obj: 1.862
Reg-MF: ite: 20; diff: 0.000 Obj: 1.862
Reg-MF: ite: 21; diff: 0.000 Obj: 1.861
Reg-MF: ite: 22; diff: 0.000 Obj: 1.861
Reg-MF: ite: 23; diff: 0.000 Obj: 1.861
Reg-MF: ite: 2

In [24]:
%%time
lambda_, k, process = 0.0001, 5, 2
mpmf = MultiprocessMF(n_user, n_item, lam=lambda_, K=k, n_process=process, tol=TOLERANCE)
mpmf.fit(train_pair, train_rating)
test_prediction = mpmf.predict(test_data)
print(f"RMSE of MultiprocessMF(lambda={lambda_}, {k=}): {rmse(test_rating, test_prediction)}")

Fitting Reg-MF: K: 5, lam: 0.00010
Fitting Reg-MF: K: 5, lam: 0.00010
Reg-MF: ite: 0; diff: 0.285 Obj: 15.677
Reg-MF: ite: 0; diff: 0.277 Obj: 15.846
Reg-MF: ite: 1; diff: 0.118 Obj: 13.835
Reg-MF: ite: 1; diff: 0.083 Obj: 14.528
Reg-MF: ite: 2; diff: 0.016 Obj: 13.612
Reg-MF: ite: 2; diff: 0.031 Obj: 14.079
Reg-MF: ite: 3; diff: 0.002 Obj: 13.589
Reg-MF: ite: 3; diff: 0.003 Obj: 14.031
Reg-MF: ite: 4; diff: 0.001 Obj: 13.578
Reg-MF: ite: 4; diff: 0.001 Obj: 14.013
Reg-MF: ite: 5; diff: 0.001 Obj: 13.571
Reg-MF: ite: 5; diff: 0.001 Obj: 14.003
Reg-MF: ite: 6; diff: 0.000 Obj: 13.565
Reg-MF: ite: 6; diff: 0.000 Obj: 13.998
Reg-MF: ite: 7; diff: 0.000 Obj: 13.562
Reg-MF: ite: 7; diff: 0.000 Obj: 13.994
Reg-MF: ite: 8; diff: 0.000 Obj: 13.559
Reg-MF: ite: 8; diff: 0.000 Obj: 13.992
Reg-MF: ite: 9; diff: 0.000 Obj: 13.557
Reg-MF: ite: 9; diff: 0.000 Obj: 13.991
Reg-MF: ite: 10; diff: 0.000 Obj: 13.556
Reg-MF: ite: 10; diff: 0.000 Obj: 13.990
Reg-MF: ite: 11; diff: 0.000 Obj: 13.554
Reg-MF:

### Side Note

Apparently, with the multiprocessing in place, things are running much faster than with only 1 process. In fact, less than half the original time. However, it appears that the model is not learning at all... This might be due to an incorrect way of updating the variables within the model among the processes.

I have spent countless hours trying an array of alternatives, including both the multiprocessing and pymp module along with shared arrays of variables, attempting at addressing the related issue, but came to no avail eventually. This is the furthest I can come to unfortunately...
